In [5]:
import sys
sys.path.append('..')
import elastic_rods
import numpy as np
from typing import NamedTuple
from bending_validation import suppress_stdout as so

In [6]:
npts = 100

In [7]:
t = np.linspace(0, 2 * np.pi, npts-2, endpoint=False)
t = np.concatenate([t, t[0:2]])

print(t)

[0.         0.06411414 0.12822827 0.19234241 0.25645654 0.32057068
 0.38468481 0.44879895 0.51291309 0.57702722 0.64114136 0.70525549
 0.76936963 0.83348377 0.8975979  0.96171204 1.02582617 1.08994031
 1.15405444 1.21816858 1.28228272 1.34639685 1.41051099 1.47462512
 1.53873926 1.60285339 1.66696753 1.73108167 1.7951958  1.85930994
 1.92342407 1.98753821 2.05165235 2.11576648 2.17988062 2.24399475
 2.30810889 2.37222302 2.43633716 2.5004513  2.56456543 2.62867957
 2.6927937  2.75690784 2.82102197 2.88513611 2.94925025 3.01336438
 3.07747852 3.14159265 3.20570679 3.26982093 3.33393506 3.3980492
 3.46216333 3.52627747 3.5903916  3.65450574 3.71861988 3.78273401
 3.84684815 3.91096228 3.97507642 4.03919055 4.10330469 4.16741883
 4.23153296 4.2956471  4.35976123 4.42387537 4.48798951 4.55210364
 4.61621778 4.68033191 4.74444605 4.80856018 4.87267432 4.93678846
 5.00090259 5.06501673 5.12913086 5.193245   5.25735913 5.32147327
 5.38558741 5.44970154 5.51381568 5.57792981 5.64204395 5.70615

In [ ]:
R = 40
pts = np.column_stack([R * np.cos(t), R * np.sin(t), np.zeros_like(t)])

In [ ]:
pr = elastic_rods.PeriodicRod(pts, True)
pr.setMaterial(elastic_rods.RodMaterial('rectangle', 4.0e10 / 1e6, 0.3, [1, 1], stiffAxis=elastic_rods.StiffAxis.D2))

In [ ]:
dc = pr.rod.deformedConfiguration()

In [ ]:
Hsp = pr.hessianSparsityPattern(1.0)

In [ ]:
import linkage_vis, vis
import matplotlib

In [ ]:
import time
def updateView():
    v.update()
    time.sleep(0.01)
    cv2.update(vectorField=np.array([d.d2 for d in dc.materialFrame]))
    refVF = vis.fields.VectorField(cv.mesh, np.array([d.d2 for d in dc.referenceDirectors]), colormap=matplotlib.cm.cool_r, glyph=vis.fields.VectorGlyph.CYLINDER)
    cv.update(vectorField=refVF)

In [ ]:
v = linkage_vis.LinkageViewer(pr, width=1200, height=720)
cv = linkage_vis.CenterlineViewer(pr.rod, superView=v, vectorField=np.array([d.d2 for d in dc.referenceDirectors]))
cv2 = linkage_vis.CenterlineViewer(pr.rod, superView=v, vectorField=np.array([d.d2 for d in dc.materialFrame]))
v.show()

In [ ]:
pr.twist = 16 * np.pi

with so(): elastic_rods.compute_equilibrium(pr, fixedVars=[pr.numDoF()-1, pr.thetaOffset(), 0, 1, 2, 3, 4, 5], callback=lambda prob, i: updateView())

# Finite Difference Validation

In [ ]:
import fd_validation

In [ ]:
class FDWrapper:
    def __init__(self, rod):
        self.rod = rod
    def numVars(self): return self.rod.numDoF()
    def getVars(self): return self.rod.getDoFs()
    def setVars(self, x): return self.rod.setDoFs(x)
    def energy(self): return self.rod.energy()
    def gradient(self): return self.rod.gradient(updatedSource=False)
    def hessian(self): return self.rod.hessian()

In [ ]:
pr.setDoFs(pr.getDoFs() + 1e-3 * fd_validation.genPerturbation(pr.getDoFs()))
pr.rod.updateSourceFrame()

In [ ]:
fd_validation.gradConvergencePlot(FDWrapper(pr))

In [ ]:
fd_validation.hessConvergencePlot(FDWrapper(pr.rod))

In [ ]:
wpr = FDWrapper(pr)
fd_validation.hessConvergencePlot(wpr, perturb=fd_validation.basisDirection(wpr, 0))